## 기상청 API 허브

In [ ]:
import sys

print("--sys.version—")
print(sys.version)

DATA 수집 목표 : 2012 ~ (최소기간)

https://apihub.kma.go.kr/

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import urllib
import urllib.request
from urllib.parse import unquote

import json

import requests
import pprint

from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

In [ ]:
import pickle

In [ ]:
with open('단기해상예보.pkl', 'wb') as f:
    pickle.dump(lst, f)

In [ ]:
with open('단기해상예보.pkl', 'rb') as f:
    lst = pickle.load(f)

In [ ]:
for i in lst:
    print(i[:5])

### Text

In [27]:
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
year2 = [2012, 2016, 2020]#, 2023]
# month = [1, 4, 7, 10]

In [26]:
def one_year_term(year):
    url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php?tm1={year}0101&tm2={year+1}0101&stn=&help=0&authKey=yEtC0d2xQmKLQtHdsWJi5g'
    response = requests.get(url)
    text = response.text.split('\n')
    return text
    

In [ ]:
def six_month_term(year, a, b):
    
    url = f'https://apihub.kma.go.kr/api/typ01/url/fct_afs_do.php?reg=&tmfc1={year}{a}0100&tmfc2={year}{b}0100&disp=1&help=0&authKey=yEtC0d2xQmKLQtHdsWJi5g'
    response = requests.get(url)
    text = response.text.split('\n')
    return text
    

In [47]:

url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php?tm1=20230101&tm2=20230313&stn=&help=0&authKey=yEtC0d2xQmKLQtHdsWJi5g'
response = requests.get(url)
text = response.text.split('\n')


In [ ]:
text[-5:]

In [10]:

url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?tm1=202301010000&tm2=202303130000&stn=&help=1&authKey=yEtC0d2xQmKLQtHdsWJi5g'
response = requests.get(url)
text = response.text.split('\n')


In [25]:
text[4:50]

['#  1. TM     : 관측시각 (KST)\r',
 '#  2. STN    : 국내 지점번호\r',
 '#  3. WD     : 풍향 (16방위)\r',
 '#  4. WS     : 풍속 (m/s)\r',
 '#  5. GST_WD : 돌풍향 (16방위)\r',
 '#  6. GST_WS : 돌풍속 (m/s)\r',
 '#  7. GST_TM : 돌풍속이 관측된 시각 (시분)\r',
 '#  8. PA     : 현지기압 (hPa)\r',
 '#  9. PS     : 해면기압 (hPa)\r',
 '# 10. PT     : 기압변화경향 (Code 0200) \r',
 '# 11. PR     : 기압변화량 (hPa)\r',
 '# 12. TA     : 기온 (C)\r',
 '# 13. TD     : 이슬점온도 (C)\r',
 '# 14. HM     : 상대습도 (%)\r',
 '# 15. PV     : 수증기압 (hPa)\r',
 '# 16. RN     : 강수량 (mm) : 여름철에는 1시간강수량, 겨울철에는 3시간강수량\r',
 '# 17. RN_DAY : 일강수량 (mm) : 해당시간까지 관측된 양(통계표)\r',
 '# 18. RN_JUN : 일강수량 (mm) : 해당시간까지 관측된 양을 전문으로 입력한 값(전문)\r',
 '# 19. RN_INT : 강수강도 (mm/h) : 관측하는 곳이 별로 없음\r',
 '# 20. SD_HR3 : 3시간 신적설 (cm) : 3시간 동안 내린 신적설의 높이\r',
 '# 21. SD_DAY : 일 신적설 (cm) : 00시00분부터 위 관측시간까지 내린 신적설의 높이\r',
 '# 22. SD_TOT : 적설 (cm) : 치우지 않고 그냥 계속 쌓이도록 놔눈 경우의 적설의 높이\r',
 '# 23. WC     : GTS 현재일기 (Code 4677)\r',
 '# 24. WP     : GTS 과거일기 (Code 4561) .. 3(황사),4(안개),5(가랑비),6(비),7(눈),8(소나기

In [28]:
lst = []

for i in tqdm(year):
    lst.append(one_year_term(i))

100%|████████████████████████████████████████████████████████████████| 11/11 [00:41<00:00,  3.76s/it]


In [ ]:
lst

In [ ]:
len(lst)

In [ ]:
# 2012, 2016, 2020 : 기간 초과

In [ ]:
lst_half = []

for i in tqdm(year2):
    lst_half.append(short_term_half(i, '01', '06'))
    lst_half.append(short_term_half(i, '07', '12'))
    

In [ ]:
len(lst_half)

In [ ]:
lst_half[4][:-5]

In [ ]:
lst[0]

In [29]:
for i in range(11):
    print(lst[i][:5])

['#START7777', '#2345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123', '# YYMMDD STN   WS    WR  WD   WS   WS  WD   WS   WS    TA    TA   TA    TA   TA    TD    TS    TG    HM    HM   HM    PV  EV_S  EV_L    FG     PA     PS     PS   PS     PS   PS   CA   SS   SS   SS    SI    SI   SI     RN     RN    RN     RN   RN     RN   RN     RN   RN     SD   SD     SD   SD    TE    TE    TE    TE    TE', '#    KST  ID  AVG   DAY MAX  MAX  MAX INS  INS  INS   AVG   MAX  MAX   MIN  MIN   AVG   AVG   MIN   AVG   MIN  MIN   AVG               DUR    AVG    AVG    MAX  MAX    MIN  MIN  TOT  DAY  DUR  CMB   DAY   60M  60M    DAY    D99   DUR    60M  60M    10M  10M    POW  POW    NEW  NEW    MAX  MAX     5    10    15    30    50', '#  

In [31]:
# columns 추출
col_name = lst[1][2].split()
# col_name = lst[1].split()
col_name = col_name[1:]
print(col_name)
print(len(col_name))

['YYMMDD', 'STN', 'WS', 'WR', 'WD', 'WS', 'WS', 'WD', 'WS', 'WS', 'TA', 'TA', 'TA', 'TA', 'TA', 'TD', 'TS', 'TG', 'HM', 'HM', 'HM', 'PV', 'EV_S', 'EV_L', 'FG', 'PA', 'PS', 'PS', 'PS', 'PS', 'PS', 'CA', 'SS', 'SS', 'SS', 'SI', 'SI', 'SI', 'RN', 'RN', 'RN', 'RN', 'RN', 'RN', 'RN', 'RN', 'RN', 'SD', 'SD', 'SD', 'SD', 'TE', 'TE', 'TE', 'TE', 'TE']
56


In [ ]:
# col_name = [i.strip() for i in col_name]

In [ ]:
# col_name[0] = 'TM_FC'

In [ ]:
col_name

In [ ]:
for i in lst:
    print(i[-3:])

In [ ]:
# df_2013, df_2014, df_2015, 
# df_2017, df_2018, df_2019, 
# df_2021, df_2022

In [ ]:
len(lst[0])

In [ ]:
for i in lst:
    print(i[-3:])

In [ ]:
# df_2013, df_2014, df_2015, 
# df_2017, df_2018, df_2019, 
# df_2021, df_2022

In [ ]:
len(lst[0])

In [37]:
lst[1][5].split()[0][:4]

'2013'

In [40]:
lst[1][5]

'20130101  90  2.5  2139  25  6.2  119  25 11.1 1432  -2.0   2.5 1352  -6.7  641 -10.6  -2.8  -8.5  52.3  38.0  118   2.8  -9.0  -9.0 -9.00 1014.7 1017.0 1019.8  248 1014.0 1326  5.1  5.4  9.6 -9.0 -9.00 -9.00   -9    0.0   -9.0  0.75   -9.0   -9   -9.0   -9   -9.0   -9   -9.0   -9   -9.0   -9 -99.0 -99.0 -99.0 -99.0 -99.0'

In [41]:
# 단기 예특보
# lst 내 text, Data Frame으로 만들기

def to_df(lst):
    for i in tqdm(range(len(lst))):
        if len(lst[i]) > 2:
            year = lst[i][5].split()[0][:4]
            globals()['df_{}'.format(year)] = pd.DataFrame(lst[i][5:-2])[0].str.split(expand=True)
#             'df_{}'.format(year).columns == col_name
#             return 'df_{}'.format(year)
#         else:
#             pass

In [42]:
to_df(lst)

100%|████████████████████████████████████████████████████████████████| 11/11 [00:05<00:00,  1.91it/s]


In [46]:
df_2015

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,20150101,90,2.2,1900,29,6.1,41,29,11.3,32,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
1,20150101,95,2.1,1777,29,4.7,1321,32,7.7,1357,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
2,20150101,98,3.4,2948,32,7.0,1043,32,10.3,1018,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
3,20150101,99,3.2,2787,29,6.2,1159,34,9.6,139,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
4,20150101,100,5.9,5128,27,9.1,1253,27,15.1,4,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34288,20160101,285,0.5,421,7,2.4,1145,5,3.7,1150,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
34289,20160101,288,0.5,471,29,2.0,1408,7,3.6,1408,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
34290,20160101,289,0.7,562,32,2.5,1634,32,5.1,1627,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
34291,20160101,294,1.6,1363,27,4.3,2343,29,8.6,2343,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0


In [ ]:
lst[1]

In [ ]:
lst[1][3].split()[0][:4]

In [ ]:
# 적설량
# lst 내 text, Data Frame으로 만들기

def to_df2(lst):
    for i in tqdm(range(len(lst))):
        year = lst[i][3].split()[0][:4]
        globals()['df_{}_snow'.format(year)] = pd.DataFrame(lst[i][3:-2])[0].str.split(expand=True)


In [ ]:
to_df2(lst)

In [ ]:
df_2022_snow

In [ ]:
text

In [ ]:
df = pd.DataFrame(lst[0][3:-2])[0].str.split(expand=True)
# df3.drop(columns = [19], inplace=True)
df.columns = col_name
df

In [ ]:
df1

In [ ]:
df_2020

In [ ]:
df[9].value_counts()

In [ ]:
'''
# 띄어쓰기에 따라 나눠진 컬럼 합치기

con_cols = [18, 19, 20, 21]

df['con_cols'] = df[con_cols].apply(lambda row : ' '.join(row.values.astype(str)), axis=1)
df.drop(columns = con_cols, inplace = True)
'''

In [ ]:
df = df.iloc[:,:11]

In [ ]:
df.drop(columns = 11, inplace=True)

In [ ]:
df_2012_snow

In [ ]:
text

In [48]:
df_2023 = pd.DataFrame(text[5:-2])[0].str.split(expand=True)
df_2023

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,20230101,90,3.0,2583,29,8.7,2,32,13.2,10,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
1,20230101,93,1.3,1113,29,5.3,1408,27,11.9,1410,...,-9,-9.0,-9,1.5,1,1.3,5.1,8.6,15.4,17.3
2,20230101,95,1.1,987,20,3.2,100,27,6.4,1401,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
3,20230101,98,1.2,1007,29,5.3,1634,29,8.5,1428,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
4,20230101,99,0.9,816,32,4.5,1353,32,7.7,1350,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6969,20230313,288,1.7,1483,34,4.9,932,36,8.0,926,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
6970,20230313,289,4.2,3604,29,9.9,339,32,18.9,316,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
6971,20230313,294,2.4,2060,29,4.4,627,25,10.3,1919,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
6972,20230313,295,2.1,1851,32,6.5,2,32,12.3,41,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0


In [ ]:
df_2023[0].sort_values()

In [50]:
# df_2013, df_2014, df_2015, df_2017, df_2018, df_2019, df_2021, df_2022

df = pd.concat([df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023])
# df1.drop(columns = [19], inplace=True)
df.columns = col_name
df

,YYMMDD,STN,WS,WR,WD,WS,WS,WD,WS,WS,...,RN,SD,SD,SD,SD,TE,TE,TE,TE,TE
0,20120101,90,2.2,1915,29,5.7,1714,29,9.8,1420,...,-9,-9.0,-9,1.6,1,-99.0,-99.0,-99.0,-99.0,-99.0
1,20120101,95,1.2,1077,34,4.8,1414,34,7.8,1407,...,-9,-9.0,-9,-9.0,-9,1.4,5.0,-99.0,-99.0,-99.0
2,20120101,98,1.8,1585,29,7.4,1445,29,11.0,1439,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
3,20120101,99,1.5,1313,29,6.0,1546,32,8.8,1607,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
4,20120101,100,4.5,3868,27,6.7,332,27,10.1,1233,...,-9,0.3,2400,22.7,2400,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6969,20230313,288,1.7,1483,34,4.9,932,36,8.0,926,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
6970,20230313,289,4.2,3604,29,9.9,339,32,18.9,316,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
6971,20230313,294,2.4,2060,29,4.4,627,25,10.3,1919,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
6972,20230313,295,2.1,1851,32,6.5,2,32,12.3,41,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0


In [ ]:
df1.reset_index(drop=True, inplace=True)
df_2023.reset_index(drop=True, inplace=True)

In [2]:
df1.to_csv('1.csv')
df_2023.to_csv('2.csv')

NameError: name 'df1' is not defined

In [3]:
df1 = pd.read_csv('./1.csv', index_col = 0)
df_2023 = pd.read_csv('./2.csv', index_col = 0)

In [4]:
df1

,YYMMDDHHMI,STN,WD,WS,GST,GST.1,GST.2,PA,PS,PT,...,TS,TE,TE.1,TE.2,TE.3,ST.1,WH,BF,IR,IX
0,201212020000,90,25,1.0,-9,-9.0,-9,1025.4,1027.7,7,...,-0.3,-99.0,-99.0,-99.0,-99.0,-9,1.5,-9,3,2
1,201212020000,95,32,0.6,-9,-9.0,-9,1007.7,1027.4,7,...,-2.2,0.7,1.2,2.3,3.3,-9,-9.0,-9,3,2
2,201212020000,98,0,0.2,-9,-9.0,-9,1012.6,1026.5,7,...,-1.9,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
3,201212020000,99,5,1.4,-9,-9.0,-9,1023.1,1026.9,7,...,-1.4,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
4,201212020000,100,0,0.1,-9,-9.0,-9,933.6,1030.5,7,...,-5.4,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748545,202301010000,285,0,0.0,-9,-9.0,-9,1026.8,1030.3,3,...,-2.6,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
748546,202301010000,288,0,0.2,-9,-9.0,-9,1028.7,1029.8,3,...,-3.8,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
748547,202301010000,289,34,1.6,-9,-9.0,-9,1013.6,1031.2,0,...,-0.9,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
748548,202301010000,294,27,1.4,-9,-9.0,-9,1024.4,1030.1,3,...,-2.8,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9


In [6]:
df_2023.columns = df1.columns

In [8]:
df = pd.concat([df1, df_2023])
# df1.drop(columns = [19], inplace=True)
# df.columns = col_name
# df.columns = df1.columns
df

,YYMMDDHHMI,STN,WD,WS,GST,GST.1,GST.2,PA,PS,PT,...,TS,TE,TE.1,TE.2,TE.3,ST.1,WH,BF,IR,IX
0,201212020000,90,25,1.0,-9,-9.0,-9,1025.4,1027.7,7,...,-0.3,-99.0,-99.0,-99.0,-99.0,-9,1.5,-9,3,2
1,201212020000,95,32,0.6,-9,-9.0,-9,1007.7,1027.4,7,...,-2.2,0.7,1.2,2.3,3.3,-9,-9.0,-9,3,2
2,201212020000,98,0,0.2,-9,-9.0,-9,1012.6,1026.5,7,...,-1.9,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
3,201212020000,99,5,1.4,-9,-9.0,-9,1023.1,1026.9,7,...,-1.4,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
4,201212020000,100,0,0.1,-9,-9.0,-9,933.6,1030.5,7,...,-5.4,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69837,202303130000,288,32,3.3,-9,-9.0,-9,1020.6,1021.6,2,...,0.7,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
69838,202303130000,289,29,9.3,29,22.0,2334,1006.4,1023.8,2,...,0.0,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
69839,202303130000,294,29,2.6,34,10.0,2322,1016.8,1022.4,2,...,1.7,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
69840,202303130000,295,32,6.1,32,12.0,2318,1017.6,1023.3,2,...,0.5,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9


In [9]:
df = df.sort_values(by='YYMMDDHHMI')
df = df.reset_index(drop=True)
df

,YYMMDDHHMI,STN,WD,WS,GST,GST.1,GST.2,PA,PS,PT,...,TS,TE,TE.1,TE.2,TE.3,ST.1,WH,BF,IR,IX
0,201212020000,90,25,1.0,-9,-9.0,-9,1025.4,1027.7,7,...,-0.3,-99.0,-99.0,-99.0,-99.0,-9,1.5,-9,3,2
1,201212020000,254,25,2.6,-9,-9.0,-9,1009.7,1025.9,7,...,-1.6,1.5,2.1,4.5,5.2,-9,-9.0,-9,3,-9
2,201212020000,253,32,1.4,-9,-9.0,-9,1019.7,1027.1,7,...,-0.1,5.3,5.1,6.3,7.1,-9,-9.0,-9,3,-9
3,201212020000,252,14,0.7,-9,-9.0,-9,1020.8,1025.6,5,...,-0.9,2.1,4.0,5.6,6.2,-9,-9.0,-9,3,-9
4,201212020000,251,0,0.4,-9,-9.0,-9,1018.3,1025.2,5,...,0.9,3.2,4.2,6.4,6.9,-9,-9.0,-9,3,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818387,202303130000,143,27,5.0,27,14.0,2237,1015.2,1022.0,2,...,0.9,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,2
818388,202303130000,140,29,6.3,29,17.0,2144,1023.2,1026.7,2,...,-0.2,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
818389,202303130000,138,25,4.4,27,12.0,2317,1019.6,1020.1,2,...,0.4,8.5,10.0,12.6,12.9,-9,0.5,-9,3,2
818390,202303130000,135,27,7.8,27,17.0,2316,992.6,1023.4,2,...,-0.5,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9


In [51]:
df.to_csv('지상관측_일.csv')

In [ ]:
df_snow = df_snow.sort_values(by='YYMMDDHHMI')
df_snow = df_snow.reset_index(drop=True)
df_snow

In [ ]:
df_snow = df_snow.drop_duplicates()
df_snow = df_snow.reset_index(drop=True)
df_snow

In [ ]:
df_snow.to_csv('적설관측(~2022).csv')

In [ ]:
df_snow = df_snow.sort_values(by='YYMMDDHHMI')
df_snow = df_snow.reset_index(drop=True)
df_snow

In [ ]:
df_snow = df_snow.drop_duplicates()
df_snow = df_snow.reset_index(drop=True)
df_snow

In [ ]:
df_snow.to_csv('적설관측(~2022).csv')

In [ ]:
df2 = pd.concat([df_2012, df_2016, df_2020])
df2.drop(columns = [19], inplace=True)
df2.columns = col_name
df2

In [ ]:
df = pd.concat([df1, df2, df3])
df.sort_values(by='TM_FC', inplace=True)
df = df.reset_index(drop=True)
df

In [ ]:
df[:20]

In [ ]:
df = df.drop_duplicates()
df

In [ ]:
df['TM_FC'].value_counts()

In [ ]:
# 필요없는 COLUMN 제거 구역코드(REG_ID), 예보관ID(MAN_ID), 예보관명(MAN_FC)
# 구간(MOD) 12, 24 시간
# SKY + PREP 한글화 = WF

df.drop(columns = ['REG_ID', 'MAN_ID', 'MAN_FC'], inplace=True)

In [ ]:
df.to_csv('단기해상예보.csv')

### XML

In [ ]:
url = 'https://apihub.kma.go.kr/api/typ02/openApi/SfcYearlyInfoService/getTyphoonList?pageNo=1&numOfRows=10&dataType=XML&year=2022&authKey=_v7wW5IUQqu-8FuSFNKrLA'

response = requests.get(url)

# xml 내용
content = response.text

# 깔끔한 출력 위한 코드
pp = pprint.PrettyPrinter(indent=4)
print(pp.pprint(content))

In [ ]:
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

#bs4 사용하여 item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('info') # item 으로 구분되어 있는 경우도 있음
print(rows)

In [ ]:
# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

In [ ]:
#xml값 DataFrame으로 만들기
xml_df = pd.DataFrame(row_list, columns=name_list)
print(xml_df.head())

In [ ]:
xml_df

In [ ]:
xml_df.to_csv('우리나라영향태풍_2021.csv')

# Data

In [ ]:
# df1 = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_1차.csv', index_col=0)
# df2 = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_2차.csv', index_col=0)
# df3 = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_3차.csv', index_col=0)

df = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보.csv', index_col=0)

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [ ]:
df3['TM_FC'] = df3['TM_FC'].astype('int64')

In [ ]:
df = pd.concat([df1, df2, df3]).reset_index()
df.drop(columns='index', inplace=True)
df

In [ ]:
df = df.sort_values(by='TM_FC', ascending=True)
df = df.reset_index(drop=True)
df

In [ ]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df

In [ ]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df

------

In [52]:
df = pd.read_csv('./지상관측_시간.csv', index_col = 0)
df2 = pd.read_csv('./지상관측_일.csv', index_col = 0)

In [55]:
df2.reset_index(drop=True, inplace = True)
df2

,YYMMDD,STN,WS,WR,WD,WS.1,WS.2,WD.1,WS.3,WS.4,...,RN.8,SD,SD.1,SD.2,SD.3,TE,TE.1,TE.2,TE.3,TE.4
0,20120101,90,2.2,1915,29,5.7,1714,29,9.8,1420,...,-9,-9.0,-9,1.6,1,-99.0,-99.0,-99.0,-99.0,-99.0
1,20120101,95,1.2,1077,34,4.8,1414,34,7.8,1407,...,-9,-9.0,-9,-9.0,-9,1.4,5.0,-99.0,-99.0,-99.0
2,20120101,98,1.8,1585,29,7.4,1445,29,11.0,1439,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
3,20120101,99,1.5,1313,29,6.0,1546,32,8.8,1607,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
4,20120101,100,4.5,3868,27,6.7,332,27,10.1,1233,...,-9,0.3,2400,22.7,2400,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387005,20230313,288,1.7,1483,34,4.9,932,36,8.0,926,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
387006,20230313,289,4.2,3604,29,9.9,339,32,18.9,316,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
387007,20230313,294,2.4,2060,29,4.4,627,25,10.3,1919,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
387008,20230313,295,2.1,1851,32,6.5,2,32,12.3,41,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0


In [56]:
df2.to_csv('지상관측_일.csv')

In [ ]:
df1.reset_index(inplace=True, drop=True)
df1

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.reset_index(inplace=True, drop=True)
df

In [ ]:
dd = pd.concat([df, df1])#.reset_index(drop=True)
# df3 = df3.reset_index(drop=True)
dd

In [ ]:
dd = dd.reset_index(drop=True)
dd

In [ ]:
type(dd)

In [ ]:
df1

In [ ]:
for i in df2['STN.1'].value_counts().index : print(i)

In [ ]:
dd.to_csv('적설관측자료.csv')